# Compteur

In [ ]:
SELECT count(*) count, TYPE
from `IT_Equipment_v3`
where STATUS is null
GROUP BY TYPE

In [ ]:
-- IT_count
select COUNT(STATUS), status
from `IT_Equipment_v3`
GROUP BY STATUS
ORDER BY COUNT(STATUS) desc

# VIEW

### `OK-POSITION : NB OF MATCH`

Version condensée de IT_Equipment + jointure avec asset_new uniquement sur la position

In [ ]:
CREATE VIEW IT_Equipment_match_position_reduced_v4
as
SELECT it.reperage_id, count(it.reperage_id) as RowNumber,
            concat(aa.asset_id) asset_id

FROM   `IT_Equipment_v3` it 
INNER JOIN `assets_filtree_v3`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
where aa.asset_id != '0'
GROUP BY it.reperage_id

In [ ]:
CREATE VIEW IT_Equipment_match_position_v4
AS
SELECT it.asset_id asset_id_it,
       it.short_name,
       it.nom,
       it.status   status_it,
       it.modele,
       it.reperage_id,
       it.hauteur_u,
       aa.ASSET_ID ASSET_ID_aa, aa.NOM_COURT NOM_COURT_aa, aa.NOM_LONG NOM_LONG_aa, aa.FABRICANT FABRICANT_aa, aa.MODEL MODEL_aa, aa.DALLE DALLE_aa, aa.SALLE SALLE_aa, aa.etat, it2.RowNumber

       
FROM   `IT_Equipment_v3` it 
INNER JOIN `assets_filtree_v3`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
INNER JOIN `IT_Equipment_match_position_reduced_v4` it2
ON it.reperage_id = it2.reperage_id

WHERE it2.rowNumber > 1
ORDER BY it2.rowNumber desc, Reperage_ID, it.status ASC

## `NOM DU FABRICANT : ASSET_ID`

### netapp & ATTO & Brocade

System 2-en-1 avec chassis + server. Y aura 2 objects differents alors que c'est la meme chose, on va donc s'assurer que le match se fait sur le chassis plutot que sur le server + on rattache tous les serveurs à au bon chassis

In [ ]:
CREATE VIEW it_built_system_NAS_v4
AS
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment_v3` it
WHERE  status IS NOT NULL and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' ) and ASSET_ID != '0'
ORDER  BY nom_de_baie DESC

### EXADATA

In [ ]:
CREATE VIEW it_build_system_EXADATA_v4
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment_v3` it
where it.fabricant like '%oracle%' and asset_id != '0'
ORDER  BY nom_de_baie DESC

### HPC

In [ ]:
CREATE VIEW it_build_system_HPC_v4
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID,
        (LEFT(Nom,6)) ROOT_FERME
FROM   `IT_Equipment_v3` it
where it.nom like '%HPC%' or it.long_name like '%HPC%'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

### HANNA

In [ ]:
CREATE VIEW it_build_system_HANNA_v4
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
        
FROM   `IT_Equipment_v3` it
where (it.nom_de_baie like '%hana%' OR it.nom like '%hana%')  and asset_id != '0'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

### RING

In [ ]:
CREATE VIEW it_build_system_RING_v4
as
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID
       
       
FROM   `IT_Equipment_v3` it
WHERE (it.short_name like '%NA9TCR%' 
OR it.nom like '%NA9TCR%') and (it.status is not null or it.asset_id != '0')	
GROUP BY nom_de_baie, nom_de_la_salle
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

### FEX

In [ ]:
CREATE VIEW IT_Equipment_FEX_v4
as
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID,
        (LEFT(it.Nom,8)) ROOT_FEX
from `IT_Equipment_v3` it
where Fabricant	 like '%cisco%' and ( it.nom = (LEFT(it.Nom,8)) )
ORDER BY Reperage_ID asc

## `'CMDB-OK-OLD'`

In [ ]:
CREATE VIEW IT_Equipment_OLD_v4
AS
SELECT it.nom_de_baie,
		aa.nom_court,
        it.short_name,
        aa.nom_long,
        it.nom,
        long_name,
       it.nom_de_la_salle,
       it.status,
       REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','') new_nom_court,
		REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','') new_nom_long
        

from `IT_Equipment_v3` it
LEFT JOIN `assets_filtree_v3` aa
ON REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name

where aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD'

## `HITASHI`

In [ ]:
CREATE VIEW IT_Equipment_cleaning_Hitachi_v4
as
SELECT it.nom_de_baie,
        it.short_name it_short_name,
        it.nom it_nom,
       it.nom_de_la_salle,
       it.status,
       (CASE WHEN LOCATE(':',it.nom) > 0 then 
       UPPER(REPLACE(REPLACE(LEFT(it.nom, LOCATE(':',it.nom)-1),'_','-'),'-',''))
       else it.nom
       END) clean_name

from `IT_Equipment_v3` it 
WHERE Fabricant like '%Hitachi%'
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

## `DXI`

In [ ]:
CREATE VIEW it_build_system_DXI_v2 as
SELECT it.asset_id asset_id_it, it.short_name, it.nom, it.status status_it, it.modele, it.reperage_id , Nom_de_la_salle, Nom_de_baie, left(it.nom,9) DXI_ROOT
FROM `IT_Equipment_v3` it 
where modele like '%dxi%' 
 and ( it.nom = left(it.nom,9) or it.short_name = left(it.nom,9) )

## ` Rack = Name`

In [ ]:
ALTER VIEW it_equipment_rack_equal_name
AS
SELECT it.modele, it.short_name, it.nom, it.asset_id, it.Nom_de_la_salle, it.nom_de_baie, it.reperage_id, it.status,
		aa.asset_id asset_id_cmdb, aa.nom_court nom_court_cmdb, aa.nom_long nom_long_cmdb, aa.rack
FROM `IT_Equipment_v3` it
INNER JOIN `assets_filtree_v3` aa
ON (it.Nom_de_baie = aa.nom_court OR it.Nom_de_baie = aa.nom_long )
WHERE it.status is null

##### View with count of unique asset id per rack / name 

In [ ]:
CREATE VIEW it_equipment_rack_equal_name_asset_count
as
select it.nom_de_baie, COUNT(DISTINCT it.asset_id_cmdb) asset_count 
from `it_equipment_rack_equal_name` it
GROUP BY it.nom_de_baie

In [ ]:
CREATE VIEW olivier_it_rack_equal_name_export
as
SELECT it.modele, it.short_name, it.nom, it.asset_id, it.Nom_de_la_salle, it.nom_de_baie, it.reperage_id, it.status, it.status_2,
		concat(it.nom_de_baie, ':', it2.asset_count) count_asset
FROM `IT_Equipment_v3` it
INNER JOIN `it_equipment_rack_equal_name_asset_count` it2
ON it.Nom_de_baie = it2.Nom_de_baie
WHERE it.status_2 is null

----------------------

# SELECT

----------------------

## QUERY 

In [ ]:
SELECT it.asset_id asset_id_it, it.short_name, it.nom, it.status status_it, it.modele, it.reperage_id, it.hauteur_u, it.status_3, it.asset_id_ok_position 
FROM `IT_Equipment_v3` it

____

### `OK-CMDB-ASSET-ID`

In [ ]:
SELECT it.asset_id asset_id_it, aa.asset_id asset_id, it.short_name, it.nom, it.status status_it, it.modele, it.reperage_id, it.hauteur_u FROM `IT_Equipment_v3` it
INNER JOIN `assets_filtree_v3` aa
ON aa.asset_id = it.asset_id
and it.asset_id != '0'

### `'ASSET_ID OK / SHORT_NAME MISMATCH'`

In [ ]:
SELECT it.asset_id asset_id_it, aa.asset_id asset_id, it.short_name, it.nom, aa.nom_court, aa.nom_long, it.modele, it.reperage_id, it.hauteur_u FROM `IT_Equipment_v3` it
INNER JOIN `assets_filtree_v3` aa
ON aa.nom_long = it.nom or (aa.nom_court = it.short_name and it.short_name !=0)
where it.status is null

### `Asset_ID affiché`

Seulement pour les serveurs, match entre les noms courts de IT_equipment et celui 
d'assets avec les 3 derniers caracteres en moins. Exemple : IT_equipment.nom = SU941, assets.nom_court = SU941AOS 

##### 0 resultat affiché, ils sont tous contenu dans le OK-CMDB-ASSET-ID

In [ ]:
SELECT it.asset_id asset_id_it, aa.asset_id asset_id, it.short_name, it.nom, aa.nom_court, aa.nom_long, it.modele, it.reperage_id, it.status FROM `IT_Equipment_v3` it
INNER JOIN `assets_filtree_v3` aa on LEFT(aa.nom_court, length(aa.nom_court) - 3) = it.short_name

where it.type = 'Serveurs' and it.status is null

### `OK-POSITION : NB OF MATCH`

HORS model qui contiennent : BL460C BL660C BL20P BL860C BL420C BL680C WS460C BL870C

In [ ]:
SELECT it.asset_id asset_id_it, aa.asset_id asset_id, it.short_name, it.nom, aa.nom_court, aa.nom_long, it.modele, aa.model, it.reperage_id, aa.dalle, it2.rownumber
FROM `IT_Equipment_v3` it 
INNER JOIN `assets_filtree_v3`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
INNER JOIN `IT_Equipment_match_position_reduced_v2` it2
ON it.reperage_id = it2.reperage_id
WHERE it.status is null
AND aa.model not like '%BL460C%'
AND aa.model not like '%BL660C%'
AND aa.model not like '%BL20P%'
AND aa.model not like '%BL860C%'
AND aa.model not like '%BL420C%'
AND aa.model not like '%BL680C%'
AND aa.model not like '%WS460C%'
AND aa.model not like '%BL870C%'
ORDER BY rownumber desc

## `NOM DU FABRICANT : ASSET_ID`

### NetApp

In [ ]:
SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, status
from `assets_filtree_v3` aa
where fabricant like 'NETAPP%'
order by dalle asc

---
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID, Nom_de_la_salle, Nom_de_baie, Slot_U, Hauteur_U
from `IT_Equipment_v3` it
where fabricant like 'NETAPP%'

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it2.status status_2,
       it.asset_id,
       it.nom, 
       it.Modele,
       it.fabricant,
       it.Reperage_ID,
       ( CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        END ) agg
       
FROM   `IT_Equipment_v3` it
 LEFT JOIN `it_built_system_NAS_v4` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

WHERE it.status is null and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' )
ORDER  BY it.nom_de_baie DESC

### Exadata

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it2.status status_2,
       it.asset_id,
       it.nom, 
       it.Modele,
       it.fabricant,
       it.Reperage_ID,
       ( CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        END ) agg
       
FROM   `IT_Equipment_v3` it
 LEFT JOIN `it_build_system_EXADATA_v4` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

WHERE it.status is null and (it.fabricant like '%oracle%')
ORDER  BY it.nom_de_baie DESC

### HPC

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it2.status status_2,
       it.asset_id,
       it.nom, 
       it.Modele,
       it.fabricant,
       it.Reperage_ID,
       ( CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.root_ferme) 
        	  WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        END ) agg
       
FROM   `IT_Equipment_v3` it
 LEFT JOIN `it_build_system_HPC_v4` it2
 ON (LEFT(it.Nom,6)) = it2.root_ferme

WHERE it.status is null and (it.fabricant like '%oracle%')
ORDER  BY it.nom_de_baie DESC

### HANNA

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it2.status status_2,
       it.asset_id,
       it.nom, 
       it.Modele,
       it.fabricant,
       it.Reperage_ID,
       ( CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        END ) agg
       
FROM   `IT_Equipment_v3` it
  LEFT JOIN `it_build_system_HANNA_v4` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

WHERE it.status is null and (it.nom_de_baie like '%hana%' OR it.nom like '%hana%')
ORDER  BY it.nom_de_baie DESC

### RING

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID,
        ( CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        END ) agg
       
       
FROM   `IT_Equipment_v3` it
LEFT JOIN `it_build_system_RING_v4` it2 
ON        it2.nom_de_baie = it.nom_de_baie 
AND       it2.nom_de_la_salle = it.nom_de_la_salle
AND		  it2.fabricant = it.fabricant
    
WHERE (it.short_name like '%NA9TCR%' 
OR it.nom like '%NA9TCR%')
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

### FEX 

#### Cette query a été lancé apres CMDB-OK-OLD et MISMATCH_TIRET

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID,
        ( CASE WHEN it.status is null and it2.status is not null and it2.asset_id !='0' then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  WHEN it.status is null and it2.status is null and (it.asset_id is null or it2.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        END ) agg
        
       
FROM   `IT_Equipment_v3` it
LEFT JOIN `IT_Equipment_FEX_v4` it2 on it2.root_fex = (LEFT(it.Nom,8)) and it.Nom != it2.root_fex
    
where it.Fabricant like '%cisco%' and it2.root_fex is not null
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

## `'CMDB-OK-OLD'`

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
       it.Reperage_ID
       
       
FROM   `IT_Equipment_v3` it
LEFT JOIN `assets_filtree_v3` aa
ON REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name
where it.status is NULL and (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD')

## `'MISMATCH_TIRET'`

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom,
       aa.nom_long,
       it.short_name,
       aa.nom_court,
        it.Modele,
        it.fabricant,
        it.Reperage_ID
        
FROM   `IT_Equipment_v3` it
INNER JOIN `assets_filtree_v3` aa
ON UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-',''))  =  UPPER(REPLACE(REPLACE(it.nom,'_',''),'-',''))
OR UPPER(REPLACE(REPLACE(aa.nom_long,'_',''),'-',''))  =  UPPER(REPLACE(REPLACE(it.nom,'_',''),'-',''))
OR UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-',''))  =  UPPER(REPLACE(REPLACE(it.short_name,'_',''),'-',''))
OR UPPER(REPLACE(REPLACE(aa.nom_long,'_',''),'-',''))  =  UPPER(REPLACE(REPLACE(it.short_name,'_',''),'-','')) 

ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle



## `HITASHI`

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       aa.asset_id,
       it.nom,
       aa.nom_long,
       it.short_name,
       aa.nom_court,
        it.Modele,
        it.fabricant,
        it.Reperage_ID,
        (
				CASE WHEN aa.asset_id is null and it.status is null then 'Hitashi : No match'
    			WHEN aa.asset_id is not null and it.status is null then concat('HDS : ',aa.asset_id)
    			END 
    			) agg
        
FROM   `IT_Equipment_v3` it
INNER JOIN `IT_Equipment_cleaning_Hitachi_v2` hit on hit.it_nom = it.nom
LEFT JOIN `assets_filtree_v3` aa on UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-','')) = hit.clean_name

WHERE it.Fabricant like '%Hitachi%'
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle